# Example of using explanation module

Run in google colaboratory

In [1]:
%cd "/Users/ainura/studies/thesis/llm-for-rec"
repo_path = "/Users/ainura/studies/thesis/llm-for-rec"

/Users/ainura/studies/thesis/llm-for-rec


Reading a Hugginface api key

In [2]:
from dotenv import load_dotenv

load_dotenv(repo_path + "/api_keys.env")

True

## Install requirements

In [3]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Add config

In [3]:
import os

config_dict = {
    "csv_args": {"delimiter": "\t"},
    "source_column": "item_id:token",
    "search_kwargs": {"k": 20},
    "data_path": os.path.join(repo_path, "datasets"),
    "load_col": {
        "inter": ["user_id", "item_id", "rating", "timestamp"],
        "item": ["item_id", "movie_title"],
    },
    "text_col": ["movie_title", "release_year", "class"],
    "MAX_ITEM_LIST_LENGTH": 10,
    "eval_args": {"split": {"LS": "valid_and_test"}, "order": "TO", "mode": "full"},
    "repeatable": True,
    "loss_type": "CE",
    "train_batch_size": 100,
    "eval_batch_size": 8,
    "valid_metric": "NDCG@10",
    "metrics": ["Recall", "NDCG"],
    "topk": [1, 20, 50, 100, 200],
    "train_neg_sample_args": None,
    "threshold_rating": 0.5,
}

## Get dataset and config

In [5]:
from llm4rec.pipelines import RecBolePipelineRecommender
from llm4rec.dataset import RecboleSeqDataset
from recbole.data.utils import data_preparation
from recbole.config import Config
from recbole.model.abstract_recommender import AbstractRecommender
import os
import torch

model_cls = RecBolePipelineRecommender
dataset_name = 'ml-100k'

config = Config(model=model_cls, dataset=dataset_name,
            config_dict=config_dict)

dataset = RecboleSeqDataset(config)
_, _, test_data = data_preparation(config, dataset)

/Users/ainura/studies/thesis/llm-for-rec/llm4rec/tasks/ranking/general_ranker.py:113: SyntaxWarning: invalid escape sequence '\d'
  candidate_match = re.search(f"\d\.\s*{candidate}", document)
/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/recbole/data/dataset/dataset.py:501: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[field].fillna(value="", inplace=True)
/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/recbole/data/dataset/dataset.py:1217: FutureWarning: using <built-in function len> 

## Let's take first user and show how explanation works in his case

Getting batch and taking 1 interaction in that batch

In [6]:
import numpy as np


i=0
batch = next(iter(test_data))
user = batch[0][i]
history_length = min(
            config["MAX_ITEM_LIST_LENGTH"],
            user['item_length']
                  )
history_ids = user['item_id_list']
history_names = np.array([test_data.dataset.item_id2text(hist_id) for hist_id in history_ids[:history_length].tolist()])
history_item_ids = test_data.dataset.id2token("item_id", user['item_id_list'][:history_length])
prev_interactions = history_item_ids

Initializing the retreival model

In [7]:
from llm4rec.tasks import RetrievalRecommender

retrieval_model = RetrievalRecommender(
                embeddings=None,
                item2text=dataset.item_token2text,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                csv_loader_args=dict(csv_args=config['csv_args'],
                                source_column=config['source_column']),
                text_splitter_args=dict(chunk_size=1000, chunk_overlap=0),
                search_type="similarity",
                search_kwargs=config['search_kwargs'],
                emb_model_kwargs={"device":"cuda:0" if torch.cuda.is_available() else "cpu"})

/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Getting candidates

In [8]:
candidates = retrieval_model.recommend(user_profile = "", prev_interactions=prev_interactions,
                                 top_k=config['search_kwargs']['k'])
candidate_ids = test_data.dataset.token2id("item_id", candidates)
candidate_texts = [test_data.dataset.item_id2text(id) for id in candidate_ids]

/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Let's recall what were the names of the items the user interacted with

In [9]:
history_names

array(['movie_title:Empire Strikes Back, The; release_year:1980; class:Action Adventure Drama Romance Sci-Fi War',
       'movie_title:Beautiful Girls; release_year:1996; class:Drama',
       'movie_title:Mars Attacks!; release_year:1996; class:Action Comedy Sci-Fi War',
       'movie_title:Broken Arrow; release_year:1996; class:Action Thriller',
       'movie_title:Amistad; release_year:1997; class:Drama',
       'movie_title:Long Kiss Goodnight, The; release_year:1996; class:Action Thriller',
       'movie_title:French Kiss; release_year:1995; class:Comedy Romance',
       'movie_title:Maltese Falcon, The; release_year:1941; class:Film-Noir Mystery',
       'movie_title:Dazed and Confused; release_year:1993; class:Comedy',
       'movie_title:Strange Days; release_year:1995; class:Action Crime Sci-Fi'],
      dtype='<U104')

And now we have suggested candidate items.

In [10]:
candidate_texts

['movie_title:Rhyme & Reason; release_year:1997; class:Documentary',
 'movie_title:Big One, The; release_year:1997; class:Comedy Documentary',
 'movie_title:Bloody Child, The; release_year:1996; class:Drama Thriller',
 'movie_title:Six Degrees of Separation; release_year:1993; class:Drama Mystery',
 'movie_title:Hunt for Red October, The; release_year:1990; class:Action Thriller',
 'movie_title:Last Supper, The; release_year:1995; class:Drama Thriller',
 'movie_title:Van, The; release_year:1996; class:Comedy Drama',
 'movie_title:The Innocent; release_year:1994; class:Drama Romance',
 'movie_title:Apostle, The; release_year:1997; class:Drama',
 'movie_title:Mimic; release_year:1997; class:Sci-Fi Thriller',
 'movie_title:Saint, The; release_year:1997; class:Action Romance Thriller',
 'movie_title:Full Monty, The; release_year:1997; class:Comedy',
 'movie_title:Twelfth Night; release_year:1996; class:Comedy Drama Romance',
 'movie_title:Something to Talk About; release_year:1995; class:C

Let's take the first item from the candidates and ask model to explain why this item can be interesting to the user

## Create an ExplainableRecommender

In [12]:
# first we need to initialize the llm, which will give us explanation
from langchain_community.llms import HuggingFaceHub
from langchain_groq import ChatGroq


# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.2",
#     task="text-generation",
#     model_kwargs={
#         "max_new_tokens": 100,
#         #"top_k": 30,
#         "temperature": 0.2,
#         "repetition_penalty": 1.05,
#     },
# )

llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)

In [13]:
from llm4rec.tasks import ExplainableRecommender


exp_recommender = ExplainableRecommender(config=config, llm=llm)

# let's try to pass first candidate and check the explanation
explanation = exp_recommender.recommend(user_interaction=user, history_names=history_names, candidate_item=candidate_texts[0])



> Entering new LLMChain chain...
Prompt after formatting:
There is a person .
The person has a list of liked items: movie_title:Empire Strikes Back, The; release_year:1980; class:Action Adventure Drama Romance Sci-Fi War, movie_title:Broken Arrow; release_year:1996; class:Action Thriller, movie_title:Amistad; release_year:1997; class:Drama, movie_title:Long Kiss Goodnight, The; release_year:1996; class:Action Thriller, movie_title:French Kiss; release_year:1995; class:Comedy Romance, movie_title:Maltese Falcon, The; release_year:1941; class:Film-Noir Mystery, movie_title:Dazed and Confused; release_year:1993; class:Comedy, movie_title:Strange Days; release_year:1995; class:Action Crime Sci-Fi. The person has a list of disliked items: movie_title:Beautiful Girls; release_year:1996; class:Drama, movie_title:Mars Attacks!; release_year:1996; class:Action Comedy Sci-Fi War. Generate ONE SENTENCE explanation why the following candidate item movie_title:Rhyme & Reason; release_year:1997; c

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



> Finished chain.


### The explanation is the following

In [14]:
explanation

"You might like the movie because it doesn't belong to the disliked genres of Drama or Action Comedy Sci-Fi War, and its documentary class is a new and unexplored genre in the liked items list."